In [19]:
%%writefile app1.py
# Importamos las librerías
import streamlit as st
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración inicial de la página
st.set_page_config(page_title="Dashboard Munich", layout="wide", page_icon="📊")

# Fondo con HTML/CSS
st.markdown("""
    <style>
        .stButton>button {
            background-color: #0066cc;
            color: white;
            font-weight: bold;
            border-radius: 5px;
            border: 2px solid transparent; /* Borde por defecto */
            transition: 0.3s;
        }

        /* Cambio de estilo al pasar el cursor */
        .stButton>button:hover {
            background-color: #004499;
            border: 2px solid black;
        }
    </style>
""", unsafe_allow_html=True)


# Cargar los datos
@st.cache_resource
def load_data():
    df = pd.read_csv("Datos_Limpios_pais_1.csv")
    return df

df = load_data()
df = df.drop(['Unnamed: 0', 'id', 'listing_url'], axis=1)

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

model_results = {} 

# Sidebar
with st.sidebar:
    View = st.selectbox(label=' Selecciona una vista', options=["Presentación", "Modelado Explicativo", "Regresión Lineal Simple", "Regresión Lineal Múltiple", "Regresión Logística"])
    if View == "Presentación":
        st.markdown("---")
        st.markdown("### 📌 Columnas del Dataset")
        st.write("**Numéricas:**", numeric_cols)
        st.write("**Categóricas:**", categorical_cols)

# ---------------- View 1: Presentación ----------------
if View == 'Presentación':
    st.title(' Dashboard de Datos de Munich')
    st.header("Presentación datos de Airbnb")
    st.markdown("#### Desarrollado por Jose Armando Ramos Ramos")

    st.image("munich1.jpg", caption="Plaza Marienplatz", use_container_width=True)

    if st.checkbox(" Mostrar datos crudos"):
        st.dataframe(df)

# ---------------- View 2: Etapa I ----------------
elif View == 'Modelado Explicativo':
    st.header(" Etapa I: Modelado Explicativo")
    st.markdown("Análisis univariado y exploración gráfica por grupos para variables categóricas.")

    tab1, tab2, tab3 = st.tabs(["📊 Análisis Univariado", "📎 Gráficas por Grupo", "📦 Barplot & Boxplot Avanzado"])

    # TAB 1: Univariado
    with tab1:
        st.subheader(" Análisis Univariado de Variables Categóricas")
        cat_var = st.selectbox(" Selecciona una variable categórica:", categorical_cols)

        st.markdown("### Resumen de la Variable")
        st.markdown(f"**Valores únicos:** {df[cat_var].nunique()}  ")
        st.markdown(f"**Ejemplos:** {df[cat_var].unique()[:5]}")

        with st.expander(" Tabla de frecuencias"):
            freq_table = df[cat_var].value_counts().nlargest(10).reset_index()
            freq_table.columns = [cat_var, "Frecuencia"]
            st.dataframe(freq_table)

        col1, col2 = st.columns(2)
        with st.sidebar:
            chart_type = st.radio("Elegir tipo de gráfico:", ["Pastel", "Barras"], key="chart_type_univ")

        if chart_type == "Pastel":
            pie_fig = px.pie(freq_table, names=cat_var, values="Frecuencia")
            col1.plotly_chart(pie_fig)
        elif chart_type == "Barras":
            bar_fig = px.bar(freq_table, x=cat_var, y="Frecuencia")
            col2.plotly_chart(bar_fig)

    # TAB 2: Gráficas por Grupo
    with tab2:
        st.subheader("📎 Gráficas por Grupo")

        group_x_tab2 = st.selectbox(" Eje X", categorical_cols, key="group_x2")
        group_hue_tab2 = st.selectbox(" Agrupador", categorical_cols + numeric_cols, key="group_hue2")

        with st.sidebar:
            st.markdown("### Mostrar gráfica agrupada (Tab 2)")
            show_grouped_bar_tab2 = st.button("Mostrar Barplot agrupado", key="btn_grouped_tab2")

        if show_grouped_bar_tab2:
            fig = px.histogram(df, x=group_x_tab2, color=group_hue_tab2, barmode="group",
                               title=f"Distribución de {group_x_tab2} por {group_hue_tab2}")
            st.plotly_chart(fig)

    # TAB 3: Avanzado - Barplot & Boxplot
    with tab3:
        st.subheader("📦 Barplot y Boxplot Avanzado")

        group_x = st.selectbox("Variable en el eje X (agrupación)", categorical_cols, key="group_x_adv")
        group_hue = st.selectbox("Variable secundaria (color o agrupador)", categorical_cols + numeric_cols, key="group_hue_adv")

        num_for_box = st.selectbox(" Variable numérica para Boxplot", numeric_cols, key="box_num_adv")

        with st.sidebar:
            st.markdown("### Elegir gráfico a mostrar (Tab 3)")
            show_bar = st.button(" Mostrar Barplot agrupado", key="btn_adv_bar")
            show_box = st.button(" Mostrar Boxplot agrupado", key="btn_adv_box")

        if show_bar:
            fig = px.histogram(df, x=group_x, color=group_hue, barmode="group",
                               title=f"Distribución de {group_x} por {group_hue}")
            st.plotly_chart(fig)

        if show_box:
            fig = px.box(df, x=group_x, y=num_for_box, color=group_x,
                         title=f"Distribución de {num_for_box} por {group_x}")
            st.plotly_chart(fig)



# ---------------- View 3: Regresión Lineal Simple ----------------
elif View == 'Regresión Lineal Simple':
    st.header("📈 View 3: Regresión Lineal Simple")

    # 1. Mapa de calor de correlación
    st.subheader("🔍 Mapa de Calor de Correlación")
    corr_matrix = df[numeric_cols].corr()
    fig_corr = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matriz de Correlación entre Variables Numéricas"
    )
    fig_corr.update_layout(width=1000, height=700)
    st.plotly_chart(fig_corr)

    # 2. Modelo de regresión lineal simple
    st.subheader("📉 Modelo de Regresión Lineal Simple")

    x_var = st.selectbox(" Variable independiente (X):", numeric_cols + categorical_cols, key="rls_x")
    y_var = st.selectbox(" Variable dependiente (Y):", numeric_cols, key="rls_y")

    if st.button("Entrenar Modelo", key="btn_rls"):
        # Si x_var es categórica, la convertimos a numérica
        if df[x_var].dtype == 'object':
            df_model = df.copy()
            df_model[x_var] = pd.factorize(df_model[x_var])[0]
        else:
            df_model = df.copy()

        X = df_model[[x_var]]
        y = df_model[y_var]

        model = LinearRegression()
        model.fit(X, y)
        pred = model.predict(X)

        # Coeficientes
        r2 = r2_score(y, pred)
        corr = pd.Series(y).corr(pd.Series(pred))  # correlación r

        st.success(f"Coeficiente de Determinación (R²): {r2:.3f}")
        st.info(f"Coeficiente de Correlación (r): {corr:.3f}")

        # Mostrar scatterplot con seaborn
        df_model["Predicciones"] = pred
        fig, ax = plt.subplots()
        sns.scatterplot(x=x_var, y=y_var, data=df_model, color="blue", label="Valores Reales", ax=ax)
        sns.scatterplot(x=x_var, y="Predicciones", data=df_model, color="red", label="Predicciones", ax=ax)
        ax.set_title(f"{y_var} vs {x_var} con predicciones")
        st.pyplot(fig)

# # ---------------- View 4: Regresión Lineal Múltiple ----------------
elif View == 'Regresión Lineal Múltiple':
    st.subheader("📉 Regresión Lineal Múltiple")

    # Mapa de calor de correlación
    st.markdown("### Mapa de Calor de Correlación")
    corr_matrix = df[numeric_cols].corr()
    fig_corr = px.imshow(
        corr_matrix,
        text_auto=True,
        color_continuous_scale="RdBu_r",
        title="Matriz de Correlación entre Variables Numéricas"
    )
    fig_corr.update_layout(width=1000, height=700)
    st.plotly_chart(fig_corr)

    # Selección de variables
    x_vars = st.multiselect(" Variables independientes:", numeric_cols, key="rlm_x")
    y_var = st.selectbox(" Variable dependiente:", numeric_cols, key="rlm_y")

    if st.button("Entrenar Modelo", key="btn_rlm") and len(x_vars) > 0:
        X = df[x_vars]
        y = df[y_var]

        model = LinearRegression()
        model.fit(X, y)
        pred = model.predict(X)

        r2 = r2_score(y, pred)
        corr = np.corrcoef(y, pred)[0, 1]

        st.success(f"Coeficiente de determinación (R²): {r2:.3f}")
        st.info(f"Coeficiente de correlación (Pearson): {corr:.3f}")

        model_results["Lineal Múltiple"] = {
            "Tipo": "Lineal",
            "Variables": x_vars,
            "Métrica": "R²",
            "R2": r2
        }

        st.markdown("### 🔎 Comparación de Reales vs. Predicciones")
        comp_df = pd.DataFrame({
            "Índice": range(len(y)),
            "Real": y.values,
            "Predicho": pred
        })

        fig = px.scatter(comp_df, x="Índice", y="Real",
                         color_discrete_sequence=["blue"],
                         labels={"y": "Valor"},
                         title="Comparación de Valores Reales (azul) y Predichos (rojo)")

        fig.add_scatter(x=comp_df["Índice"], y=comp_df["Predicho"],
                        mode='markers',
                        marker=dict(color='red'),
                        name='Predicho')

        fig.update_layout(xaxis_title="Índice", yaxis_title="Valor")
        st.plotly_chart(fig)

# ---------------- View 5: Regresión Logística ----------------
elif View == 'Regresión Logística':
    st.subheader("📊 Regresión Logística")
    target = st.selectbox("🔺 Variable binaria objetivo:", categorical_cols)

    all_features = numeric_cols + categorical_cols
    features = st.multiselect("🔸 Variables independientes:", all_features)

    if st.button("Entrenar Modelo", key="btn_log") and len(features) > 0:
        df_filtered = df[df[target].notna()]
        y = (df_filtered[target] == df_filtered[target].unique()[0]).astype(int)

        # One-hot encoding si hay categóricas
        X = pd.get_dummies(df_filtered[features], drop_first=True)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        acc = accuracy_score(y_test, pred)
        precision = precision_score(y_test, pred)
        recall = recall_score(y_test, pred)
        cm = confusion_matrix(y_test, pred)

        st.success(f"Accuracy: {acc:.3f} | Precision: {precision:.3f} | Recall (Sensibilidad): {recall:.3f}")

        model_results["Logística"] = {"Tipo": "Logística", "Variables": features, "Métrica": "Accuracy", "R2": acc}

        st.markdown("###  Matriz de Confusión")
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
        st.pyplot(fig)

        comp_df = pd.DataFrame({"Real": y_test, "Predicho": pred})
        fig2 = px.scatter(comp_df, x="Real", y="Predicho", title="Predicción de Clase")
        st.plotly_chart(fig2)


Overwriting app1.py
